# Convolutional Neural Networks for Text Classification
## Andy Kreek
## July 11, 2017

# What will we cover?

* Setup and Notation (barf)
* Example data: IMDB movie reviews
* Word vectors
* 1D convolutions
* Max pooling
* A convolutional neural network for text classification
* Visualizing our results

# The General Setup

Our data set consists of $N$ pairs $\{(s_i, y_i)\}_{i=1,\ldots,N}$.

* $s_i$ are sequences of tokens: $s_i=[s_{i1}, s_{i2}, ..., s_{il_i}]$ where $l_i$ is the length of the $i$-th sequence.
* $y_i$ are labels.

For example, $s_{2533}$ could be `["the", "cat", "went", "outside"]`.

Sometimes, the sequences are padded with a special `<UNK>` token or truncated so that they all have the same length.

__Example__

* Pad to length 7: `["<UNK>", "<UNK>", "<UNK>", "the", "cat", "went", "outside"]`
* Truncate to length 3: `["the", "cat", "went"]`

## Vocabulary

The set of unique tokens is $V = \{t_j\}$, aka the vocabulary. The size of the vocabulary is $|V|$.

By convention, index $j$ corresponds to the $j$-th most frequent token.

Index $0$ is typically reserved for the `<UNK>` token.

# The Data

![imdb logo](http://ia.media-imdb.com/images/M/MV5BMTk3ODA4Mjc0NF5BMl5BcG5nXkFtZTgwNDc1MzQ2OTE@._V1_.png)

* 50000 highly polarized movie reviews, compiled in 2011 at Stanford.
* 25000 train / 25000 test.
* In each set, 12500 positive (7/10 rating or better) and 12500 negative (4/10 or worse).

[Maas, et al. (2011)](http://www.aclweb.org/anthology/P11-1015)

### Positive Example

(Emphasis added)

>This is a __really interesting movie__. It is an action movie with comedy mixed in. Foxx teams up with comedian Epps in this movie to give it a comedic spin. It will keep you wondering whats going to happen to Foxx next. It was a __well shot movie__, the director __used the right colors__ in this movie(dark blue colors) to give it the right kind of feel. Kimberly Elise also starred in this movie and it is __always a pleasure__ to see her on the big screen. She __plays her role well__. Even Jamie Kennedy is in this movie. __It's worth seeing it__ you haven't seen it. [...]

### Negative Example

(Emphasis added)

>This had to be one of the __worst films ever__. When Kate shows up and Jed is with a bunch of guys and they all start clapping...so Hollywood. __Another bad scene__ was when Kate was running her hands over the pavement it appeared that she was examining the texture of the pavement. Andie's acting is so bad in this film - __I could not connect with them__ or feel there love for each other. The other 2 women however were very good and overshadowed Andie. Anyway this was __such a campy movie__. [...]

# Word Vectors


## 1-Hot Encoding

Let $x_{t_j} = (0,\ldots,0,1,0,\ldots,0) \in \mathbf{R}^{|V|}$. (The $j$-th entry of $x_{t_j}$ is 1, and the rest are 0)

![one-hot example](img/one_hot_vectors.svg)

### Is this actually useful?

* 1-hot vectors are sparse and high dimensional
* Semantic/syntactic relationships are lost.

![issues with one-hot encoding](img/equidistant_word_vectors.svg)

## Dense Word Embeddings

Can we create a more useful representation of the words in our vocabulary?

Ideally, we want:

* Low dimenstional vectors.
* Semantic/syntactic relationships between words are preserved.

Word2Vec, developed by [Mikolov et al. (2013a)](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf), is a popoular neural network model that uses word adjacency to achieve these goals.

GloVe, developed by [Pennington et al. (2014)](https://nlp.stanford.edu/pubs/glove.pdf) at Stanford, is another.

So, instead of this...

![one-hot vectors](img/one_hot_vectors.svg)

We have this!

![dense word vectors](img/dense_embedding.svg)




And, relationships between words are encoded in their vector-space embeddings!

![queen king](img/mikolov_2013b.png)

[Mikolov et al (2013b)](https://www.aclweb.org/anthology/N/N13/N13-1090.pdf)

## Code Time!

We'll be using [Keras](https://keras.io) to build our CNN.

Here's how to add word embeddings to a Keras model.

In [7]:
# This model "stub" is nothing more than a word-embedding layer
# with randomly initialized weights.

import numpy as np
from keras.models import Sequential
from keras.layers import Embedding

embedding_example = Sequential()
embedding_example.add(
    Embedding(input_length=6,
              input_dim=15,
              output_dim=5))
    
# There is also a `weights` kwarg for pretrained word vectors.

In [8]:
# Instead of sequences of tokens to our model,
# we just pass the indices in our vocabulary.

# One record is one sequence of token indices.
x = np.array([[0,1,2,3,4,5]])
embedding_example.predict(x)

array([[[-0.0168216 , -0.02629953, -0.04406124,  0.0296027 ,  0.02183444],
        [-0.03417917,  0.01199464, -0.01235622, -0.02431607, -0.02335169],
        [ 0.01407507, -0.03061602, -0.01962676,  0.01174899, -0.0052474 ],
        [ 0.02606836, -0.0067421 , -0.00047936,  0.01490219, -0.01871777],
        [-0.04216436,  0.04351366, -0.00182826, -0.0266427 , -0.0264061 ],
        [-0.0361799 , -0.03256632,  0.02672929, -0.00998612,  0.04182993]]], dtype=float32)

# 1D Convolutions

#### con·vo·lu·tion
/ˌkänvəˈlo͞oSHən/ _noun_
1. a coil or twist, especially one of many.
2. __MATHEMATICS__ a function derived from two given functions by integration that expresses how the shape of one is modified by the other.

## Filters

a.k.a filter maps, kernels

(Since there is a 1-to-1 correspondence of tokens to embedding vectors, with dimension $d$, we'll use $s_{ij}$ to refer to the embedding vector of the $j$-th word of the $i$-th sequence in our data set, not its token.)

A convolutional filter $W$ is an $k \times d$ set of weights ($k$ is called the width of the filter).

$$ W = 
\begin{pmatrix}
| & \ldots & | \\
w_1 & \ldots & w_k \\ 
| & \ldots & | \\
\end{pmatrix}
= 
\begin{pmatrix}
w_{11} & \ldots & w_{1k} \\
\vdots & \ddots & \vdots \\
w_{d1} & \ldots & w_{dk} \\
\end{pmatrix}
$$

## Filters

![convolution](img/convolution.svg)

Now, just imagine each cell represents a vector, and that's all there is to it...

Oh, and add a bias term and pass the convolution through an activation function, tyically non-linear.

## Code Time!

In [9]:
from keras.layers import Conv1D

# Initialize our hard-coded convolutional filter
conv_weights = np.zeros((3,5,1))
conv_weights[0,:,0] = 1
conv_bias = np.zeros(1)

# A keras stub, just like before
conv_example = Sequential()
conv_example.add(
    Conv1D(input_shape=(6,5),
           activation='relu',
           filters=1, # typically, you have many of these
           kernel_size=3,
           weights=[conv_weights, conv_bias]))

In [10]:
# Remember, this represents a single record.
# Each innermost vector is a word embedding
x=np.array(
    [[[ 0.1, 0.8, 0.3, 0.6, 0.3],
      [ 0.9, 0.1, 0.8, 0.6, 0.8],
      [ 0.6, 0.9, 0.1, 0.6, 0.8],
      [ 0.4, 0.1, 0.4, 0.8, 0.5],
      [ 0.7, 0.3, 0.0, 0.1, 0.3],
      [ 0.7, 0.6, 0.8, 0.9, 0.1]]])
conv_example.predict(x)

array([[[ 2.10000014],
        [ 3.20000029],
        [ 3.00000024],
        [ 2.20000005]]], dtype=float32)

# Max Pooling

![max pool](img/max_pool.svg)

Really, that's it.

If there is more than one convolutional filter, take the max over each one.

## Code Time!

In [14]:
from keras.layers import GlobalMaxPooling1D

# The simplest one yet. No weights!
max_pool_example = Sequential()
max_pool_example.add(GlobalMaxPooling1D(input_shape=(10,4)))

In [16]:
x = np.random.random((1,10,4))
print("x =", x)
max_pool_example.predict(x)

x = [[[ 0.02805121  0.56714374  0.68598691  0.91093192]
  [ 0.30906004  0.51941898  0.24748326  0.27194739]
  [ 0.66433485  0.35545886  0.14589052  0.50848656]
  [ 0.88815581  0.14820827  0.37789361  0.09558142]
  [ 0.97695208  0.55932136  0.29546261  0.91124264]
  [ 0.61772018  0.20485886  0.46668867  0.55264841]
  [ 0.51788783  0.88604824  0.97008294  0.12199895]
  [ 0.48131472  0.59174507  0.26781702  0.22640531]
  [ 0.94715823  0.53648895  0.16410375  0.81017351]
  [ 0.37258971  0.50133634  0.66927668  0.64511177]]]


array([[ 0.97695208,  0.88604826,  0.97008294,  0.91124266]], dtype=float32)

# Bonus: Fully Connected Feed-Forward Network

#### Enough said

![fully connected](img/fully_connected.png)

From [stats.stackexchange](https://stats.stackexchange.com/questions/234891/difference-between-convolution-neural-network-and-deep-learning/234894)

# CNN for Text Classification

We are now ready to assemble our building blocks.

## Network Architecture

![1-layer cnn](img/1_layer_cnn.svg)

(Inspired by [Kim 2014](http://www.aclweb.org/anthology/D14-1181))

## Hyperparameters

* Embedding Layer
    - Embedding dimension
    - Pre-trained weights
* Convolutional Layer
    - Number of filters
    - Filter width(s)
    - Activation function
* Fully-Connected Layer(s)
    - Number of layers
    - Number of nodes per layer
    - Activation function
* General
    - Regularization
    - Dropout

## Code Time!

In [21]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 25000
maxlen = 150 # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)#[:5000, :]
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)#[:5000, :]
y_train = y_train#[:5000]
y_test = y_test#[:5000]
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 150)
x_test shape: (25000, 150)


In [47]:
from keras.layers import Dropout, Dense, Flatten
from keras.regularizers import l2

cnn = Sequential()

cnn.add(Embedding(input_length=150,
              input_dim=25000,
              output_dim=50))
cnn.add(Dropout(0.2))

cnn.add(Conv1D(activation='relu',
               filters=15,
               kernel_size=5,
               kernel_regularizer=l2(0.0003),
               bias_regularizer=l2(0.0003),
               name='conv_layer'))

cnn.add(GlobalMaxPooling1D(name='flat_max_pool'))
cnn.add(Dropout(0.2))

In [48]:
cnn.add(Dense(units=15,
              kernel_regularizer=l2(0.0003),
              bias_regularizer=l2(0.0003),
              activation='relu'))
cnn.add(Dropout(0.2))

cnn.add(Dense(units=1,
              kernel_regularizer=l2(0.0003),
              bias_regularizer=l2(0.0003),
              activation='sigmoid'))

cnn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [49]:
cnn_hist = cnn.fit(x_train, y_train,
                   batch_size=64,
                   epochs=2,
                   validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.6107 - acc: 0.6536 - val_loss: 0.4394 - val_acc: 0.8086

# Visualizing the Results

## T-SNE Plot

A technique for dimensionality reduction.

(I had some technical difficulties... so here is another example )

![tsne plot](https://d3ansictanv2wj.cloudfront.net/images/digits_tsne-5d3f7058.png)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras.models import Model

%matplotlib inline

# New model that chops off the dense/output layers from our CNN.
encoding = Model(inputs=cnn.input,
                 outputs=cnn.get_layer('flat_max_pool').output)

# Subsample the data
idxs = np.random.choice(x_train.shape[0], 3000, replace=False)

# Compute review encoding
encodings = encoding.predict(x_train[idxs,:])

# Create TSNE plot
tsne = TSNE(n_components=2, init='pca', random_state=0, n_iter=4000)
x_tsne = tsne.fit_transform(encodings)

h = plt.scatter(x_tsne[:,0], x_tsne[:,1], 
                c=y_train[idxs],
                #c=cnn.predict(x_test[idxs,:]),
                alpha=0.4,
                cmap=plt.cm.viridis)
#g.get_figure().savefig('test_tsne.png')
plt.title('TSNE Plot: File-Note Encodings w/ Predictions')
plt.colorbar()

## Convolutional Filter Activations

If we print the text of our reviews colored according to the value of the activation for our convolutional layer at each token, we can get a sense for what each filter is looking for.

In [ ]:
# Create mapping of indices to words
word_index = imdb.get_word_index()
index_word = {a:b for b,a in word_index.items()}
index_word[0] = '-'

In [52]:
# Make a new model that just gets the activations from the convolution layer
convolution = Model(inputs=cnn.input,
                    outputs=cnn.get_layer('conv_layer').output)

convolutions = convolution.predict(x_test)

In [123]:
import matplotlib as mpl
import matplotlib.colors as cl
from IPython.display import display, HTML

def text_and_probabilities_to_html(text, activations, cmap):
    words = text.split(' ')
    return ''.join([span_template(w, p, cmap) for w, p in zip(words, activations)])
        
def span_template(token, pred, cmap):
    span = ('<span style="background:' +
            cl.rgb2hex(cmap(pred)) + 
            '; color:' + 
            cl.rgb2hex(plt.cm.gray(pred)) + 
            '"> ' +
            token + 
            ' </span>')
    return span

def visualize_classifier(text, activations):
    cmap = plt.cm.Blues
    #norm = mpl.colors.Normalize(vmin=np.min(activations), vmax=np.max(activations))
    html_text = text_and_probabilities_to_html(text, activations, cmap)
    print(html_text)
    display(HTML(html_text))

In [101]:
convolutions = convolution.predict(x_test)

In [127]:
n = 6186
filter_id = 0
sample_text = ' '.join([index_word[max(0, i-3)] for i in x_test[n,:]])

activations=convolutions[n,:,filter_id]
activations = np.hstack([activations, np.zeros(4)])

visualize_classifier(sample_text, activations)

<span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="b

## A Negative Review

#### A Negative Filter
<span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> spoiler </span><span style="background:#c7dbef; color:#3f3f3f"> with </span><span style="background:#82bbdb; color:#717171"> plot </span><span style="background:#f7fbff; color:#000000"> explanation </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#eaf3fb; color:#101010"> poor </span><span style="background:#f7fbff; color:#000000"> family </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> being </span><span style="background:#f7fbff; color:#000000"> thrown </span><span style="background:#f7fbff; color:#000000"> in </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> middle </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> war </span><span style="background:#f7fbff; color:#000000"> between </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> evil </span><span style="background:#f7fbff; color:#000000"> guy </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#d0e1f2; color:#333333"> pops </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> from </span><span style="background:#f7fbff; color:#000000"> nowhere </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> no </span><span style="background:#f7fbff; color:#000000"> questions </span><span style="background:#f7fbff; color:#000000"> asked </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> went </span><span style="background:#f7fbff; color:#000000"> wacko </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> casper </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> friendly </span><span style="background:#f7fbff; color:#000000"> ghost </span><span style="background:#f7fbff; color:#000000"> trying </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> avenge </span><span style="background:#f7fbff; color:#000000"> his </span><span style="background:#f7fbff; color:#000000"> death </span><span style="background:#f7fbff; color:#000000"> with </span><span style="background:#f7fbff; color:#000000"> every </span><span style="background:#f7fbff; color:#000000"> frame </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> gets </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#a6cee4; color:#595959"> more </span><span style="background:#b4d3e9; color:#4e4e4e"> predictable </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#e4eff9; color:#181818"> big </span><span style="background:#4e9acb; color:#969696"> disappointment </span><span style="background:#d0e1f2; color:#333333"> ending </span><span style="background:#f7fbff; color:#000000"> seams </span><span style="background:#cbdef1; color:#393939"> completely </span><span style="background:#f7fbff; color:#000000"> dumb </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> has </span><span style="background:#f7fbff; color:#000000"> nothing </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> with </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> itself </span><span style="background:#f7fbff; color:#000000"> i </span><span style="background:#f7fbff; color:#000000"> think </span><span style="background:#f7fbff; color:#000000"> someone </span><span style="background:#f7fbff; color:#000000"> run </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> ideas </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> just </span><span style="background:#f7fbff; color:#000000"> get </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> point </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> s </span><span style="background:#f7fbff; color:#000000"> t </span><span style="background:#f7fbff; color:#000000"> this </span><span style="background:#f7fbff; color:#000000"> way </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> other </span><span style="background:#f7fbff; color:#000000"> so </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> may </span><span style="background:#f7fbff; color:#000000"> as </span><span style="background:#c1d9ed; color:#444444"> well </span><span style="background:#1c6bb0; color:#c5c5c5"> have </span><span style="background:#549fcd; color:#919191"> a </span><span style="background:#cde0f1; color:#363636"> stupid </span><span style="background:#e6f0f9; color:#161616"> ending </span><span style="background:#c3daee; color:#424242"> waste </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> time </span><span style="background:#f7fbff; color:#000000"> resources </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> money </span><span style="background:#8dc1dd; color:#6a6a6a"> actor </span><span style="background:#75b4d8; color:#797979"> play </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> very </span><span style="background:#f7fbff; color:#000000"> poor </span><span style="background:#f7fbff; color:#000000"> if </span><span style="background:#f7fbff; color:#000000"> you </span><span style="background:#f7fbff; color:#000000"> think </span><span style="background:#f7fbff; color:#000000"> about </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> dialogs </span><span style="background:#f7fbff; color:#000000"> you </span><span style="background:#f7fbff; color:#000000"> may </span><span style="background:#f7fbff; color:#000000"> find </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> family </span><span style="background:#f7fbff; color:#000000"> conflict </span><span style="background:#eaf2fb; color:#111111"> is </span><span style="background:#d0e2f2; color:#323232"> artificially </span><span style="background:#f7fbff; color:#000000"> enforced </span><span style="background:#f7fbff; color:#000000"> by </span><span style="background:#cde0f1; color:#363636"> poor </span><span style="background:#f7fbff; color:#000000"> communication </span><span style="background:#f7fbff; color:#000000"> between </span><span style="background:#f7fbff; color:#000000"> characters </span><span style="background:#f7fbff; color:#000000"> some </span><span style="background:#f7fbff; color:#000000"> scenes </span><span style="background:#f5f9fe; color:#030303"> - </span><span style="background:#eaf3fb; color:#101010"> random </span><span style="background:#add0e6; color:#545454"> in </span><span style="background:#083776; color:#f8f8f8"> summary </span><span style="background:#08306b; color:#ffffff"> boring </span><span style="background:#08306b; color:#ffffff"> predictable </span><span style="background:#4594c7; color:#9d9d9d"> poor </span><span style="background:#9fcae1; color:#5f5f5f"> pathetic </span><span style="background:#a6cee4; color:#595959"> waste </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> time </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> money </span>

## A Negative Review

#### Another Filter (What is this one looking for?)
<span style="background:#f5fafe; color:#020202"> - </span><span style="background:#e9f2fa; color:#121212"> - </span><span style="background:#e3eef8; color:#1a1a1a"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> spoiler </span><span style="background:#f7fbff; color:#000000"> with </span><span style="background:#f7fbff; color:#000000"> plot </span><span style="background:#f7fbff; color:#000000"> explanation </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#f7fbff; color:#000000"> poor </span><span style="background:#f7fbff; color:#000000"> family </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> being </span><span style="background:#f7fbff; color:#000000"> thrown </span><span style="background:#f7fbff; color:#000000"> in </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> middle </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> war </span><span style="background:#f7fbff; color:#000000"> between </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> evil </span><span style="background:#f7fbff; color:#000000"> guy </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> pops </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> from </span><span style="background:#f7fbff; color:#000000"> nowhere </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> no </span><span style="background:#f7fbff; color:#000000"> questions </span><span style="background:#f7fbff; color:#000000"> asked </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> went </span><span style="background:#f7fbff; color:#000000"> wacko </span><span style="background:#f4f9fe; color:#040404"> and </span><span style="background:#f7fbff; color:#000000"> casper </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> friendly </span><span style="background:#f7fbff; color:#000000"> ghost </span><span style="background:#f7fbff; color:#000000"> trying </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> avenge </span><span style="background:#f7fbff; color:#000000"> his </span><span style="background:#f7fbff; color:#000000"> death </span><span style="background:#f7fbff; color:#000000"> with </span><span style="background:#f5f9fe; color:#030303"> every </span><span style="background:#f7fbff; color:#000000"> frame </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> gets </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> predictable </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#f7fbff; color:#000000"> big </span><span style="background:#f7fbff; color:#000000"> disappointment </span><span style="background:#f7fbff; color:#000000"> ending </span><span style="background:#f7fbff; color:#000000"> seams </span><span style="background:#f7fbff; color:#000000"> completely </span><span style="background:#f7fbff; color:#000000"> dumb </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> has </span><span style="background:#f7fbff; color:#000000"> nothing </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> with </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> itself </span><span style="background:#f7fbff; color:#000000"> i </span><span style="background:#f7fbff; color:#000000"> think </span><span style="background:#f7fbff; color:#000000"> someone </span><span style="background:#f7fbff; color:#000000"> run </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> ideas </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> just </span><span style="background:#f7fbff; color:#000000"> get </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> point </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> movie </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> s </span><span style="background:#f7fbff; color:#000000"> t </span><span style="background:#f7fbff; color:#000000"> this </span><span style="background:#f7fbff; color:#000000"> way </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> other </span><span style="background:#cee0f2; color:#353535"> so </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> may </span><span style="background:#f7fbff; color:#000000"> as </span><span style="background:#eef5fc; color:#0c0c0c"> well </span><span style="background:#f7fbff; color:#000000"> have </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#f7fbff; color:#000000"> stupid </span><span style="background:#f7fbff; color:#000000"> ending </span><span style="background:#f7fbff; color:#000000"> waste </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> time </span><span style="background:#f7fbff; color:#000000"> resources </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> money </span><span style="background:#e2edf8; color:#1b1b1b"> actor </span><span style="background:#f7fbff; color:#000000"> play </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> very </span><span style="background:#f7fbff; color:#000000"> poor </span><span style="background:#f7fbff; color:#000000"> if </span><span style="background:#f7fbff; color:#000000"> you </span><span style="background:#f7fbff; color:#000000"> think </span><span style="background:#f7fbff; color:#000000"> about </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> dialogs </span><span style="background:#f7fbff; color:#000000"> you </span><span style="background:#f7fbff; color:#000000"> may </span><span style="background:#f7fbff; color:#000000"> find </span><span style="background:#f7fbff; color:#000000"> out </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f1f7fd; color:#080808"> family </span><span style="background:#f2f8fd; color:#060606"> conflict </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> artificially </span><span style="background:#f7fbff; color:#000000"> enforced </span><span style="background:#f7fbff; color:#000000"> by </span><span style="background:#f7fbff; color:#000000"> poor </span><span style="background:#f7fbff; color:#000000"> communication </span><span style="background:#f7fbff; color:#000000"> between </span><span style="background:#f7fbff; color:#000000"> characters </span><span style="background:#f7fbff; color:#000000"> some </span><span style="background:#f7fbff; color:#000000"> scenes </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> random </span><span style="background:#f7fbff; color:#000000"> in </span><span style="background:#f7fbff; color:#000000"> summary </span><span style="background:#f7fbff; color:#000000"> boring </span><span style="background:#f7fbff; color:#000000"> predictable </span><span style="background:#f7fbff; color:#000000"> poor </span><span style="background:#f7fbff; color:#000000"> pathetic </span><span style="background:#f7fbff; color:#000000"> waste </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> time </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> money </span>


## A Positive Review

#### The same negative filter from before
<span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> this </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> definitely </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> most </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> thought </span><span style="background:#f7fbff; color:#000000"> provoking </span><span style="background:#f7fbff; color:#000000"> piece </span><span style="background:#f7fbff; color:#000000"> not </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> say </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#e7f1fa; color:#141414"> memento </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> insomnia </span><span style="background:#f7fbff; color:#000000"> are </span><span style="background:#f7fbff; color:#000000"> bad </span><span style="background:#f7fbff; color:#000000"> but </span><span style="background:#f7fbff; color:#000000"> they </span><span style="background:#f7fbff; color:#000000"> were </span><span style="background:#f7fbff; color:#000000"> definitely </span><span style="background:#f7fbff; color:#000000"> up </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> hollywood </span><span style="background:#f7fbff; color:#000000"> standards </span><span style="background:#f7fbff; color:#000000"> while </span><span style="background:#f7fbff; color:#000000"> following </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> an </span><span style="background:#f7fbff; color:#000000"> indie </span><span style="background:#f7fbff; color:#000000"> flick </span><span style="background:#f7fbff; color:#000000"> the </span><span style="background:#f7fbff; color:#000000"> story </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> very </span><span style="background:#f7fbff; color:#000000"> brilliant </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> very </span><span style="background:#f7fbff; color:#000000"> well </span><span style="background:#f7fbff; color:#000000"> developed </span><span style="background:#f7fbff; color:#000000"> overall </span><span style="background:#f7fbff; color:#000000"> watch </span><span style="background:#f7fbff; color:#000000"> this </span><span style="background:#f7fbff; color:#000000"> if </span><span style="background:#f7fbff; color:#000000"> your </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#f7fbff; color:#000000"> fan </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> any </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> work </span><span style="background:#f7fbff; color:#000000"> i'm </span><span style="background:#f7fbff; color:#000000"> sure </span><span style="background:#f7fbff; color:#000000"> you'll </span><span style="background:#f7fbff; color:#000000"> be </span><span style="background:#f7fbff; color:#000000"> able </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> appreciate </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> more </span>


## A Positive Review

#### The same other filter... it appears to be looking for more positive phrases
<span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#f5fafe; color:#020202"> - </span><span style="background:#e9f2fa; color:#121212"> - </span><span style="background:#e6f0f9; color:#161616"> - </span><span style="background:#edf4fc; color:#0d0d0d"> - </span><span style="background:#d9e7f5; color:#272727"> - </span><span style="background:#d6e5f4; color:#2b2b2b"> - </span><span style="background:#f6faff; color:#010101"> this </span><span style="background:#eef5fc; color:#0c0c0c"> is </span><span style="background:#e7f1fa; color:#141414"> definitely </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> most </span><span style="background:#f7fbff; color:#000000"> and </span><span style="background:#f7fbff; color:#000000"> thought </span><span style="background:#f7fbff; color:#000000"> provoking </span><span style="background:#f7fbff; color:#000000"> piece </span><span style="background:#f7fbff; color:#000000"> not </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> say </span><span style="background:#f7fbff; color:#000000"> that </span><span style="background:#f7fbff; color:#000000"> memento </span><span style="background:#f7fbff; color:#000000"> or </span><span style="background:#f7fbff; color:#000000"> insomnia </span><span style="background:#f7fbff; color:#000000"> are </span><span style="background:#f7fbff; color:#000000"> bad </span><span style="background:#f7fbff; color:#000000"> but </span><span style="background:#f7fbff; color:#000000"> they </span><span style="background:#f7fbff; color:#000000"> were </span><span style="background:#f7fbff; color:#000000"> definitely </span><span style="background:#f7fbff; color:#000000"> up </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> hollywood </span><span style="background:#f7fbff; color:#000000"> standards </span><span style="background:#f7fbff; color:#000000"> while </span><span style="background:#f7fbff; color:#000000"> following </span><span style="background:#f7fbff; color:#000000"> is </span><span style="background:#f7fbff; color:#000000"> more </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> an </span><span style="background:#f7fbff; color:#000000"> indie </span><span style="background:#f7fbff; color:#000000"> flick </span><span style="background:#7db8da; color:#747474"> the </span><span style="background:#ccdff1; color:#383838"> story </span><span style="background:#ddeaf7; color:#212121"> is </span><span style="background:#2b7bba; color:#b6b6b6"> very </span><span style="background:#cee0f2; color:#353535"> brilliant </span><span style="background:#e2edf8; color:#1b1b1b"> and </span><span style="background:#c3daee; color:#424242"> very </span><span style="background:#e6f0f9; color:#161616"> well </span><span style="background:#f7fbff; color:#000000"> developed </span><span style="background:#f7fbff; color:#000000"> overall </span><span style="background:#f7fbff; color:#000000"> watch </span><span style="background:#f7fbff; color:#000000"> this </span><span style="background:#f7fbff; color:#000000"> if </span><span style="background:#f7fbff; color:#000000"> your </span><span style="background:#f7fbff; color:#000000"> a </span><span style="background:#f7fbff; color:#000000"> fan </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> any </span><span style="background:#f7fbff; color:#000000"> of </span><span style="background:#f7fbff; color:#000000"> - </span><span style="background:#f7fbff; color:#000000"> work </span><span style="background:#f7fbff; color:#000000"> i'm </span><span style="background:#f7fbff; color:#000000"> sure </span><span style="background:#f7fbff; color:#000000"> you'll </span><span style="background:#f7fbff; color:#000000"> be </span><span style="background:#f7fbff; color:#000000"> able </span><span style="background:#f7fbff; color:#000000"> to </span><span style="background:#f7fbff; color:#000000"> appreciate </span><span style="background:#f7fbff; color:#000000"> it </span><span style="background:#f7fbff; color:#000000"> more </span>

## Most Active Text

How about if we find the snippet of text that _most_ excites each filter?

1. Compute the activations from the convolution layer.
2. Find the reviews in which the top-$m$ activations occur.
3. In each of the top-$m$ reviews, find the position of the maximal activation.
4. Extract the $k$-gram from the review starting at that position.

In [128]:
# for each convolutional filter, get the top 10 test examples
for idx in range(15):
    most_active_row_indices = np.argsort(convolutions[:,:,idx].max(axis=1))[-10:]
    most_active_window_indices = convolutions[most_active_row_indices,:,idx].argmax(axis=1)
    print('\nMost active windows for filter {}'.format(idx))
    for r, w in zip(most_active_row_indices, most_active_window_indices):
        window = ' '.join([index_word[max(0,v-3)] for v in x_test[r, w:(w+5)]])
        print('({:.2}) {}'.format(convolutions[r,w,idx],window))


Most active windows for filter 0
(1.6) horrible terrible sadly pathetic excuse
(1.7) worst rob - worst worst
(1.7) just horrible horrible horrible waste
(1.7) gets worst and worst what's
(1.7) plot worst acting worst special
(1.8) awful awful awful awful awful
(1.8) awful awful awful awful awful
(1.9) this is the worst waste
(1.9) waste hole the worst script
(2.1) boring predictable poor pathetic waste

Most active windows for filter 1
(0.65) and works very very well
(0.66) it works very very well
(0.66) is touching and very well
(0.68) overall it's a very well
(0.68) journey and a very well
(0.68) beautiful is also very well
(0.69) time is captured very well
(0.7) notch very tense very well
(0.7) music is also very well
(0.71) very brilliant and very well

Most active windows for filter 2
(0.69) equally superb eastwood is great
(0.69) is flawless pitch is perfect
(0.71) very inspiring and uplifting highly
(0.71) captures beautiful picture perfect images
(0.71) music is simply superb 

### Most active windows for filter 0

* (1.6) horrible terrible sadly pathetic excuse
* (1.7) worst rob - worst worst
* (1.7) just horrible horrible horrible waste
* (1.7) gets worst and worst what's
* (1.7) plot worst acting worst special
* (1.8) awful awful awful awful awful
* (1.8) awful awful awful awful awful
* (1.9) this is the worst waste
* (1.9) waste hole the worst script
* (2.1) boring predictable poor pathetic waste

### Most active windows for filter 12

* (0.85) still enjoy it definitely recommended
* (0.87) overall good time highly recommended
* (0.88) paced and exciting highly recommended
* (0.9) worth a viewing highly recommended
* (0.9) truly moving film highly recommended
* (0.92) 2005 it's a highly recommended
* (0.92) wonderfully br br highly recommended
* (0.94) site is also highly recommended
* (0.95) this one is highly recommended
* (1.0) inspiring and uplifting highly recommended

### Most active windows for filter 5

* (1.5) awful awful awful drab unimaginative
* (1.6) terrible cgi awful script stupid
* (1.6) worst plot worst acting worst
* (1.6) worse than waste 85 minutes
* (1.6) horrible acting horrible script horrible
* (1.6) horrible acting horrible script horrible
* (1.7) just horrible horrible horrible waste
* (1.8) poor pathetic waste of time
* (1.9) awful awful awful awful awful
* (1.9) awful awful awful awful awful

... wait a minute. Are different filters just learning the same things?